# SQL research

Research objectives:

Analyze the database. The database contains information about books, publishers, authors, as well as user reviews of books.

Among the tasks: to analyze the number of books published since 2000, to identify the most discussed and most highly rated books, to identify the publisher with the largest number of published books, to determine the most highly rated author and overall user activity on the platform.

## Examining tables

Let's check the contents of the tables in the database.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import text, create_engine

# configuring db connection
db_config = {'user': '...', # username
'pwd': '...', # password
'host': '...',
'port': 1111, # port
'db': '....'} # db name
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# connector
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# query
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [3]:
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [4]:
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Tasks

**Let's calculate how many books were published after January 1, 2000.**

In [6]:
query = '''SELECT COUNT(DISTINCT(book_id))
           FROM books 
           WHERE (CAST(publication_date AS DATE)) > '2000-01-01';'''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


В базе данных после 1 января 2000 года вышло 819 книг.

**For each book, let's count the number of reviews and the average rating**

In [7]:
query = '''SELECT b.book_id,
                  b.title,
                  COUNT(DISTINCT(r.review_id)),
                  AVG(rt.rating)
           FROM books AS b
           LEFT JOIN reviews AS r ON r.book_id = b.book_id
           LEFT JOIN ratings AS rt ON rt.book_id = b.book_id
           GROUP BY b.book_id, b.title
           ORDER BY COUNT(r.review_id) DESC;'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.662500
1,750,The Hobbit or There and Back Again,6,4.125000
2,673,The Catcher in the Rye,6,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


There are only 1000 books in the database. The most discussed book (with the most reviews) is Twilight (book 1).

**Let's determine the publishing house that has released the largest number of books thicker than 50 pages**

In [8]:
query = '''SELECT p.publisher,
                  COUNT(DISTINCT(b.book_id))
           FROM publishers AS p
           LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY COUNT(DISTINCT(b.book_id)) DESC
           LIMIT 1;'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


The publisher with the most published books (not counting publishers that produce pamphlets) is Penguin Books.

**Determine the author with the highest average book rating - considering only books with 50 or more ratings**

In [9]:
query = '''
            WITH 
            i AS (SELECT b.title AS title,
                         b.book_id AS book_id,
                         a.author AS author,
                         COUNT(r.rating_id) AS ratings,
                         AVG(r.rating) AS rating_avg
                  FROM books AS b
                  INNER JOIN ratings AS r ON b.book_id = r.book_id
                  INNER JOIN authors AS a ON b.author_id = a.author_id
                  GROUP BY title, b.book_id, a.author
                  HAVING COUNT(r.rating_id) >= 50
                  ORDER BY rating_avg DESC
                  )
            SELECT i.author,
                   AVG(i.rating_avg) AS rating
            FROM i 
            GROUP BY author
            ORDER BY rating DESC
            LIMIT 1;'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844


The author with the highest average rating for his works (among popular books - with the number of ratings of 50 or more) is J. Rowling.

**Let's calculate the average number of reviews from users who have given more than 48 ratings**

In [10]:
query = '''
            WITH 
            i AS (SELECT rt.username AS user,
                         COUNT(rt.rating_id) AS number_of_ratings
                  FROM ratings AS rt
                  GROUP BY rt.username
                  HAVING COUNT(rt.rating_id) > 48
                  )
            SELECT AVG(count) 
            FROM (SELECT i.user,
                   COUNT(r.review_id)
            FROM i
            JOIN reviews as r ON i.user = r.username
            GROUP BY i.user) AS counts;'''

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


The most active users (those who gave more than 48 ratings) left an average of 24 reviews. Thus, active users leave ratings at least twice as often as reviews.

**Final conclusions:**

- There are only 1000 books in the database. Of these, more than 80% have been published since 2000.


- The most discussed book (with the most reviews) is Twilight (book 1), while the author of the Harry Potter works, J.K. Rowling, has the highest average rating for his works among popular authors.


- The publisher with the largest number of published books (excluding publishers that produce brochures) - Penguin Books.


- Even the most active users leave reviews at least twice as often as ratings.